In [1]:
births = spark.table("births_2018")
display(births)
births.columns

NameError: name 'spark' is not defined

In [2]:
# wybranie kolumn gdzie bmi > 0
data = (
  births
    .where(births['bmi'] > 0)
)
print("Liczba wierszy: %d" % (data.count()))


Liczba wierszy: 3801534

In [3]:
# usunięcie kolumn z pustymi wartościami
births_data = data.dropna(how='any')
print("Liczba usuniętych pusty wierszy: ", data.count() - births_data.count())
print("Liczba pozostałych wierszy:", births_data.count())

Liczba usuniętych pusty wierszy: 666965
Liczba pozostałych wierszy: 3134569

In [4]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler

string_columns = ['ip_gon', 'mm_aicu', 'sex', 'ld_indl', 'mtran', 'rf_cesar', 'rf_cesarn']
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(births_data) for column in string_columns]

vector_assembler = VectorAssembler(inputCols = list(set(births_data.columns) - set(string_columns) - set(['bmi'])), outputCol = 'features')\


pipeline = Pipeline(stages=indexers+[vector_assembler])
births_data = pipeline.fit(births_data).transform(births_data).select(['features', 'bmi'])

In [5]:
# podstawowe statystyki dla danych
births_data.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
bmi,3134569,28.759007827286588,12.483797151774885,13.0,99.9


In [6]:
splits = births_data.randomSplit([0.7, 0.3])
trainingData = splits[0]
testData = splits[1]

In [7]:
trainingData.describe().show()

+-------+------------------+
summary| bmi|
+-------+------------------+
 count| 2194005|
 mean| 28.7578893418117|
 stddev|12.483044396530628|
 min| 13.0|
 max| 99.9|
+-------+------------------+

In [8]:
#budowa modelu regresji liniowej 
from pyspark.ml.regression import LinearRegression
linear = LinearRegression(featuresCol = "features", labelCol='bmi', maxIter=10)
linear_model = linear.fit(trainingData)
print("Wspolczynniki: " + str(linear_model.coefficients))
print("Wyraz wolny: " + str(linear_model.intercept))

trainingSummary = linear_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

Wspolczynniki: [0.008776286432631822,-0.011637108128352532,-0.00024499324010087895,-0.21561172296356512,-0.12504533040448998,-0.009460520205417537,0.005955769753027569,0.13059796779392616,0.05204020497934182,-0.2271141675876309,-0.16301844411141692,-0.31707620328617453,0.17941401048588462,0.3703702619910818,-0.008582513166886274,0.1402857499058081,0.05154734019798334,0.024691639861312408,-0.00011107939913102131,0.056195601199978046,8.249304624031634e-05,0.0003755516272133685,0.0003154552923210158,-0.0845368457599498,0.00018243470824012167,0.006648273489882272,-0.042175021675902126,0.07335395638376234,0.011485650713139646,0.019958199558790333,0.001986457365394682,0.09425258668561698,-0.6590252162267692,0.014227144094411063,-0.002452084115286937,-0.028199491755175592,-1.704617091864481e-06,0.00019640868543561391,-0.006465132188382236,0.05732587250187496,-2.2456893528187983e-05,0.01368062216309355,-0.0023239572668661227]
Wyraz wolny: -20.849732983290053
RMSE: 4.424276
r2: 0.874384

In [9]:
from pyspark.ml.evaluation import RegressionEvaluator
#Testowanie na danych testowych
linear_predictions = linear_model.transform(testData)
linear_predictions.select("prediction","bmi","features").show(5)
linear_evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="bmi",metricName="r2")
print("R Squared (R2) dla danych testowych = %g" % linear_evaluator.evaluate(linear_predictions))
test_result = linear_model.evaluate(testData)
print("Root Mean Squared Error (RMSE) dla danych testowych = %g" % test_result.rootMeanSquaredError)

+------------------+----+--------------------+
 prediction| bmi| features|
+------------------+----+--------------------+
22.671688958443873|22.4|[1.0,0.0,1.0,1.0,...|
25.180374380098797|17.8|[1.0,0.0,1.0,1.0,...|
20.238002473863823|18.5|[1.0,0.0,1.0,1.0,...|
25.321538801474453|25.6|[1.0,0.0,1.0,1.0,...|
 30.03439760098476|31.9|[1.0,0.0,1.0,1.0,...|
+------------------+----+--------------------+
only showing top 5 rows

R Squared (R2) dla danych testowych = 0.873055
Root Mean Squared Error (RMSE) dla danych testowych = 4.44852

In [10]:
# Drzewa decyzyjne
from pyspark.ml.regression import DecisionTreeRegressor
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer

# Wykrycie zmiennych kategorycznyc, jezeli jest 5 lub mniej różnych wartości dla kolumny to traktujemy
# ją jako zmienną kategoryczną
featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=5).fit(births_data)
decisionTreeReg = DecisionTreeRegressor(featuresCol ='indexedFeatures', labelCol = 'bmi')
pipeline = Pipeline(stages=[featureIndexer, decisionTreeReg])

decisionTreeReg_model = pipeline.fit(trainingData)

In [11]:
#Testowanie na danych testowych
dt_predictions = decisionTreeReg_model.transform(testData)
print(dt_predictions)
rmse = RegressionEvaluator(labelCol="bmi", predictionCol="prediction", metricName="rmse").evaluate(dt_predictions)
r2 = RegressionEvaluator(predictionCol="prediction", labelCol="bmi", metricName="r2").evaluate(dt_predictions)
dt_predictions.select("prediction", "bmi", "features").show(5)
print("Root Mean Squared Error (RMSE) na danych testowych = %g" % rmse)
print("R Squared (R2) dla danych testowych = %g" % r2)
# summary only
print(decisionTreeReg_model)

DataFrame[features: vector, bmi: float, indexedFeatures: vector, prediction: double]
+------------------+----+--------------------+
 prediction| bmi| features|
+------------------+----+--------------------+
 19.75095651762363|22.4|[1.0,0.0,1.0,1.0,...|
 19.75095651762363|17.8|[1.0,0.0,1.0,1.0,...|
 19.75095651762363|18.5|[1.0,0.0,1.0,1.0,...|
 27.6065725961597|25.6|[1.0,0.0,1.0,1.0,...|
34.075847804657066|31.9|[1.0,0.0,1.0,1.0,...|
+------------------+----+--------------------+
only showing top 5 rows

Root Mean Squared Error (RMSE) na danych testowych = 2.77274
R Squared (R2) dla danych testowych = 0.950682
PipelineModel_654fd8f9deb8